# Delete databases, tables and partitions and data

`pydbtools` now has functions to remove databases, database tables, and partitions, plus the underlying data on S3.

## Setup

First run the following cells to set up the database tables.

Import the necessary libraries.

In [ ]:
import os
import pandas as pd
import awswrangler as wr
import pydbtools as pydb

Set up your testing area. 

**Important:** substitute your own Github username below.

In [ ]:
# setup your own testing area (set foldername = GH username)
foldername = "mratford"  # GH username
foldername = foldername.lower().replace("-", "_")

In [ ]:
bucketname = "alpha-everyone"
s3_base_path = f"s3://{bucketname}/{foldername}/"

db_name = f"aws_example_{foldername}"
source_db_base_path = f"s3://{bucketname}/{foldername}/source_db/"

# Delete all the s3 files in a given path
if wr.s3.list_objects(s3_base_path):
    print("deleting objs")
    wr.s3.delete_objects(s3_base_path)

# Delete the database if it exists
df_dbs = wr.catalog.databases(None)
if db_name in df_dbs["Database"].to_list():
    print(f"{db_name} found deleting")
    wr.catalog.delete_database(name=db_name)

# Setup source database
# Create the database
wr.catalog.create_database(db_name)

# Iterate through the tables in data/ and write them to our db using awswrangler
for table_name in ["department", "employees"]:
    df = pd.read_csv(f"data/{table_name}.csv")
    table_path = os.path.join(source_db_base_path, f"{table_name}/")
    wr.s3.to_parquet(
        df=df,
        path=table_path,
        index=False,
        dataset=True,  # True allows the other params below i.e. overwriting to db.table
        database=db_name,
        table=table_name,
        mode="overwrite",
    )

# For the sales table partition the data by employee_id and qtr
# (reduce the size of the table for legibility)
df = pd.read_csv("data/sales.csv").query("employee_id < 5")
table_path = os.path.join(source_db_base_path, "sales")
partition_info = wr.s3.to_parquet(
    df=df,
    path=table_path,
    index=False,
    dataset=True,
    partition_cols=["employee_id", "qtr"],
    database=db_name,
    table="sales",
    mode="overwrite",
)

## Deleting a table

Show the tables in the database.

In [ ]:
table_info = list(wr.catalog.get_tables(database=db_name))
[x["Name"] for x in table_info]

Show the data for the `department` table.

In [ ]:
dept_info = next(x for x in table_info if x["Name"] == "department")
dept_location = dept_info["StorageDescriptor"]["Location"]
wr.s3.list_objects(dept_location)

Now delete the `department` table.

In [ ]:
pydb.delete_table_and_data(database=db_name, table="department")

Check that it's no longer in the database.

In [ ]:
table_info = list(wr.catalog.get_tables(database=db_name))
[x["Name"] for x in table_info]

Check that the data no longer exists.

In [ ]:
wr.s3.list_objects(dept_location)

## Deleting a partition

Show the partitions from the `sales` table.

In [ ]:
wr.catalog.get_partitions(database=db_name, table="sales")

Check the data for one of the partitions.

In [ ]:
wr.s3.list_objects(f"{source_db_base_path}sales/employee_id=1/qtr=4/")

Use an SQL like query to delete the partition and data for quarter 4.

In [ ]:
pydb.delete_partitions_and_data(database=db_name, table="sales", expression="qtr = 4")

Check that the partition no longer exists.

In [ ]:
wr.catalog.get_partitions(database=db_name, table="sales")

Check that the data no longer exists.

In [ ]:
wr.s3.list_objects(f"{source_db_base_path}sales/employee_id=1/qtr=4/")

Using a more complex query, delete quarters 1 and 2 for employee 3.

In [ ]:
pydb.delete_partitions_and_data(
    database=db_name, table="sales", expression="employee_id = 3 and qtr < 3"
)
wr.catalog.get_partitions(database=db_name, table="sales")

See [the documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/glue.html#Glue.Client.get_partitions) for details on the expression syntax.

## Deleting a database

In [ ]:
db_name in list(wr.catalog.databases()["Database"])

In [ ]:
pydb.delete_database_and_data(db_name)

In [ ]:
db_name in list(wr.catalog.databases()["Database"])

In [ ]:
wr.s3.list_objects(source_db_base_path)

In [ ]:
source_db_base_path

## Deleting temporary database tables

It might be useful during development to get rid of the temporary database or one of it's tables if something has gone wrong. This can be accomplished by using `__temp__` as the database name in one of the functions above.

In [ ]:
# Setup source database
# Create the database
wr.catalog.create_database(db_name, exist_ok=True)

# Iterate through the tables in data/ and write them to our db using awswrangler
for table_name in ["department", "employees", "sales"]:

    df = pd.read_csv(f"data/{table_name}.csv")
    table_path = os.path.join(source_db_base_path, f"{table_name}/")
    wr.s3.to_parquet(
        df=df,
        path=table_path,
        index=False,
        dataset=True,  # True allows the other params below i.e. overwriting to db.table
        database=db_name,
        table=table_name,
        mode="overwrite",
    )

sql = f"""
SELECT employee_id, sum(sales) as total_sales
FROM {db_name}.sales
GROUP BY employee_id
"""
print(sql)
pydb.create_temp_table(sql, table_name="total_sales")

In [ ]:
pydb.read_sql_query("select * from __temp__.total_sales")

In [ ]:
pydb.delete_database_and_data("__temp__")

In [ ]:
try:
    df = pydb.read_sql_query("select * from __temp__.total_sales")
    print("Error, temporary database not deleted correctly.")
except wr.exceptions.QueryFailed:
    print("Query failed correctly.")